## Cleaning

In [1]:
import pandas as pd
IG_LOG = 'parsed_data/parsed_ig_dms.csv'

ig_df = pd.read_csv(IG_LOG)

/var/folders/8k/cwqdxf015ds_r60lk_3g6vv40000gn/T/ipykernel_46954/3377192495.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# filter out start call messages
ig_df[ig_df['content'].apply(lambda x: 'started an audio call' not in x if isinstance(x, str) else True)]

# make a separate df for calls
ig_calls = ig_df[ig_df['call_duration'].notna()]
ig_df = ig_df[ig_df['call_duration'].isna()]

In [3]:
# make a separate df for shared content
ig_media = ig_df[ig_df['content'].apply(lambda x: 'sent an attachment.' in x if isinstance(x, str) else False)]
ig_df = ig_df[ig_df['content'].apply(lambda x: 'sent an attachment.' not in x if isinstance(x, str) else True)]

ig_media = pd.concat([ig_media, ig_df[ig_df['share_link'].notna()]], ignore_index=True)
ig_df = ig_df[ig_df['share_link'].isna()]

ig_media = pd.concat([ig_media, ig_df[ig_df['share_content_owner'].notna()]], ignore_index=True)
ig_df = ig_df[ig_df['share_content_owner'].isna()]

In [4]:
ig_msgs = ig_df[['sender_name', 'timestamp_ms', 'content', 'reaction']]
ig_calls = ig_calls[['sender_name', 'timestamp_ms', 'content', 'call_duration']]
ig_media = ig_media[['sender_name', 'timestamp_ms', 'content', 'share_link', 'share_text', 'share_content_owner', 'reaction']]

In [5]:
ig_msgs.to_csv('intermediate_data/ig_msgs.csv', index=False)
ig_calls.to_csv('intermediate_data/ig_calls.csv', index=False)
ig_media.to_csv('intermediate_data/ig_media.csv', index=False)